In [ ]:
import numpy as np
import os
import trimesh
import meshio
from noise import pnoise3
import tetgen as tg
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go
from concurrent.futures import ThreadPoolExecutor


In [ ]:

class HeightmapMeshGenerator:
    def __init__(self, size_x=100, size_y=100, amplitude=1.0, noise_function=None, **noise_params):
        """Initialize the generator with given parameters."""
        self.size_x = size_x
        self.size_y = size_y
        self.amplitude = amplitude
        self.noise_function = noise_function if noise_function is not None else self.fbm_noise
        self.noise_params = noise_params

        self.heightmap = None
        self.vertices = None
        self.faces = None
        self.mesh_vertices = None
        self.mesh_faces = None
        self.tetrahedral_mesh = None

        # Create a Plotly figure widget for visualization
        self.fig_widget = go.FigureWidget()

    # Noise Functions

    def fbm_noise(self, x, y, z=0, octaves=4, persistence=0.5, lacunarity=2.0):
        """Fractal Brownian Motion noise."""
        value = 0.0
        amplitude = 1.0
        frequency = 1.0
        for _ in range(octaves):
            value += amplitude * pnoise3(x * frequency, y * frequency, z * frequency)
            amplitude *= persistence
            frequency *= lacunarity
        return value

    def perlin_noise(self, x, y, z=0):
        """Simple Perlin noise."""
        return pnoise3(x, y, z)

    def square_wave(self, x, y, z=0, frequency=10.0, amplitude=1.0):
        """Square wave noise."""
        return amplitude * np.sign(np.sin(frequency * x))
    
    def sine_wave(self, x, y, z=0, frequency=10.0, amplitude=1.0):
        """Sine wave noise."""
        return amplitude * np.sin(frequency * x)

    def beckmann_noise(self, x, y, alpha=0.5):
        """Beckmann microfacet distribution."""
        tan_theta_h = np.sqrt(x**2 + y**2) / alpha
        cos_theta_h = 1 / np.sqrt(1 + tan_theta_h**2)
        D = np.exp(-(tan_theta_h**2)) / (np.pi * alpha**2 * cos_theta_h**4)
        return D

    def ggx_noise(self, x, y, alpha=0.5):
        """GGX (Trowbridge-Reitz) microfacet distribution."""
        tan2_theta_h = (x**2 + y**2) / alpha**2
        cos_theta_h = 1 / np.sqrt(1 + tan2_theta_h)
        cos2_theta_h = cos_theta_h**2
        D = alpha**2 / (np.pi * ((cos2_theta_h * (alpha**2 - 1) + 1)**2))
        return D

    def blinn_noise(self, x, y, n=20):
        """Blinn-Phong microfacet distribution."""
        tan_theta_h = np.sqrt(x**2 + y**2)
        cos_theta_h = 1 / np.sqrt(1 + tan_theta_h**2)
        D = (n + 2) / (2 * np.pi) * cos_theta_h**n
        return D

    def mandelbrot_noise(self, x, y, L=1.0, D_f=1.5, gamma=1.2, M=10, N_max=10, max_iterations=100, threshold=2.0):
        """
        Ausloos-Berman Weierstrass-Mandelbrot fractal noise function.
        Generates a fractal height value at coordinates (x, y) using the formula:

        y(x) = L^(4 - 2*D_f) ln(γ) * Σ_m=1^M Σ_n=1^N_max [γ^(n-1)(2(D_f-2)) * 
                (cos(2πγ^(n-1)x/L - cos(πm/M) + φ_mn)]

        Parameters:
            x, y (float): Spatial coordinates for evaluation.
            L (float): Length scale of the noise.
            D_f (float): Fractal dimension (affects self-similarity scaling).
            gamma (float): Frequency scaling factor.
            M (int): Number of angular divisions (outer summation index).
            N_max (int): Maximum frequency index (inner summation index).
            max_iterations (int): Maximum number of iterations for the Mandelbrot set.
            threshold (float): Threshold for the Mandelbrot set.
        Returns:
            float: Fractal noise value at (x, y).
        """
        # Precompute scaling factors
        log_gamma = np.log(gamma)
        scale_factor = L**(4 - 2 * D_f) * log_gamma

        # Precompute angles for M
        thetas = np.pi * np.arange(1, M + 1) / M

        # Use a fixed random seed for reproducibility
        rs = np.random.RandomState(42)

        noise_value = 0.0
        # use threaded for loop
        for n in range(1, N_max + 1):
            # Frequency and amplitude scaling
            freq_scale = gamma**(n - 1)
            amp_scale = freq_scale**(2 * (D_f - 2))

            for m in range(1, M + 1):
                # Random phase for (m, n)
                phi_mn = rs.uniform(0, 2 * np.pi)
                # Compute projection and combined cosine terms
                term1 = np.cos(2 * np.pi * freq_scale * x / L)
                term2 = np.cos(thetas[m - 1])
                term3 = phi_mn
                noise_value += amp_scale * (term1 - term2 + term3)

        return scale_factor * noise_value

    # Heightmap Generation
    def generate_heightmap(self):
        """Generate the heightmap using the selected noise function."""
        size_x, size_y = self.size_x, self.size_y
        x = np.linspace(-1, 1, size_x)
        y = np.linspace(-1, 1, size_y)
        x_grid, y_grid = np.meshgrid(x, y, indexing='ij')

        # Check if noise function is one that cannot be vectorized easily
        # (like fbm_noise, perlin_noise, and mandelbrot_noise)
        non_vectorized = [self.fbm_noise, self.perlin_noise, self.mandelbrot_noise]
        if self.noise_function in non_vectorized:
            heightmap = np.zeros((size_x, size_y))
            for i in range(size_x):
                for j in range(size_y):
                    xi = x[i]
                    yj = y[j]
                    heightmap[i, j] = self.noise_function(xi, yj, **self.noise_params)
        else:
            # Vectorized functions
            exponent = self.noise_params.get('exponent', 1.0)
            noise_params_no_exponent = {k: v for k, v in self.noise_params.items() if k != 'exponent'}
            D = self.noise_function(x_grid, y_grid, **noise_params_no_exponent)
            D_transformed = D ** exponent
            heightmap = D_transformed

        # Normalize and scale the heightmap
        heightmap = self.normalize_heightmap(heightmap) * self.amplitude
        self.heightmap = heightmap

    def normalize_heightmap(self, heightmap):
        """Normalize the heightmap to range [0, 1]."""
        min_val = np.min(heightmap)
        max_val = np.max(heightmap)
        if max_val - min_val == 0:
            raise ValueError("Heightmap has no variation; normalization is undefined.")
        return (heightmap - min_val) / (max_val - min_val)

    # Mesh Generation and Processing

    def heightmap_to_closed_mesh(self):
        """Convert the heightmap to a closed 3D mesh."""
        size_x, size_y = self.heightmap.shape

        # Create grid of indices
        i = np.arange(size_x)
        j = np.arange(size_y)
        ii, jj = np.meshgrid(i, j, indexing='ij')

        # Flatten the grid indices
        ii_flat = ii.flatten()
        jj_flat = jj.flatten()
        height_flat = self.heightmap.flatten()

        # Generate top and bottom vertices
        top_vertices = np.column_stack((ii_flat, jj_flat, height_flat))
        bottom_vertices = np.column_stack((ii_flat, jj_flat, np.zeros_like(height_flat)))

        # Combine vertices
        vertices = np.vstack((top_vertices, bottom_vertices))

        # Index offset for bottom vertices
        offset = size_x * size_y

        # Create vertex indices grid
        vertex_indices = np.arange(size_x * size_y).reshape((size_x, size_y))
        vertex_indices_bottom = vertex_indices + offset

        # Indices for cells (quads)
        v0 = vertex_indices[:-1, :-1]
        v1 = vertex_indices[1:, :-1]
        v2 = vertex_indices[:-1, 1:]
        v3 = vertex_indices[1:, 1:]

        # Flatten the indices
        v0_flat = v0.flatten()
        v1_flat = v1.flatten()
        v2_flat = v2.flatten()
        v3_flat = v3.flatten()

        # Create faces for the top surface
        faces_top = np.vstack([
            np.column_stack([v0_flat, v1_flat, v2_flat]),
            np.column_stack([v1_flat, v3_flat, v2_flat])
        ])

        # Bottom surface faces (reverse the order to flip normals)
        v0b = vertex_indices_bottom[:-1, :-1]
        v1b = vertex_indices_bottom[1:, :-1]
        v2b = vertex_indices_bottom[:-1, 1:]
        v3b = vertex_indices_bottom[1:, 1:]

        v0b_flat = v0b.flatten()
        v1b_flat = v1b.flatten()
        v2b_flat = v2b.flatten()
        v3b_flat = v3b.flatten()

        faces_bottom = np.vstack([
            np.column_stack([v0b_flat, v2b_flat, v1b_flat]),
            np.column_stack([v1b_flat, v2b_flat, v3b_flat])
        ])

        # Side faces
        # Left side (j=0)
        v_top_left = vertex_indices[:, 0]
        v_bot_left = vertex_indices_bottom[:, 0]

        faces_left = np.vstack([
            np.column_stack([v_top_left[:-1], v_bot_left[:-1], v_bot_left[1:]]),
            np.column_stack([v_top_left[:-1], v_bot_left[1:], v_top_left[1:]])
        ])

        # Right side (j=size_y-1)
        v_top_right = vertex_indices[:, -1]
        v_bot_right = vertex_indices_bottom[:, -1]

        faces_right = np.vstack([
            np.column_stack([v_top_right[:-1], v_bot_right[1:], v_bot_right[:-1]]),
            np.column_stack([v_top_right[:-1], v_top_right[1:], v_bot_right[1:]])
        ])

        # Front side (i=0)
        v_top_front = vertex_indices[0, :]
        v_bot_front = vertex_indices_bottom[0, :]

        faces_front = np.vstack([
            np.column_stack([v_top_front[:-1], v_bot_front[:-1], v_bot_front[1:]]),
            np.column_stack([v_top_front[:-1], v_bot_front[1:], v_top_front[1:]])
        ])

        # Back side (i=size_x-1)
        v_top_back = vertex_indices[-1, :]
        v_bot_back = vertex_indices_bottom[-1, :]

        faces_back = np.vstack([
            np.column_stack([v_top_back[:-1], v_bot_back[1:], v_bot_back[:-1]]),
            np.column_stack([v_top_back[:-1], v_top_back[1:], v_bot_back[1:]])
        ])

        # Combine all faces
        faces = np.vstack((faces_top, faces_bottom, faces_left, faces_right, faces_front, faces_back))

        self.vertices = vertices
        self.faces = faces

    def repair_mesh(self):
        """Repair the mesh if it's not watertight."""
        mesh = trimesh.Trimesh(vertices=self.vertices, faces=self.faces)
        if not mesh.is_watertight:
            # Attempt to repair the mesh
            trimesh.repair.fill_holes(mesh)
            trimesh.repair.fix_normals(mesh)
            trimesh.repair.fix_inversion(mesh)
            # Remove duplicate faces or vertices
            mesh.remove_duplicate_faces()
            mesh.remove_duplicate_vertices()
            mesh.remove_unreferenced_vertices()
            # Check again if the mesh is watertight
            if not mesh.is_watertight:
                print("Warning: Mesh is still not watertight after repair.")
                # Try to repair again
                trimesh.repair.fill_holes(mesh)
                trimesh.repair.fix_normals(mesh)
                trimesh.repair.fix_inversion(mesh)
                mesh.remove_duplicate_faces()
                mesh.remove_duplicate_vertices()
                mesh.remove_unreferenced_vertices()
                if not mesh.is_watertight:
                    raise RuntimeError("Mesh is not watertight after repair.")
        self.mesh_vertices = mesh.vertices
        self.mesh_faces = mesh.faces

    def simplify_mesh(self, target_face_count):
        """Simplify the mesh to a target number of faces."""
        mesh = trimesh.Trimesh(vertices=self.mesh_vertices, faces=self.mesh_faces)
        simplified_mesh = mesh.simplify_quadratic_decimation(target_face_count)
        self.mesh_vertices = simplified_mesh.vertices
        self.mesh_faces = simplified_mesh.faces
        print(f"Simplified to {len(self.mesh_faces)} faces.")

    def save_mesh_to_stl(self, filename):
        """Save the mesh to an STL file."""
        meshio_mesh = meshio.Mesh(points=self.mesh_vertices, cells=[("triangle", self.mesh_faces)])
        meshio.write(filename, meshio_mesh)
        print(f"Mesh saved to {filename}")

    def tetrahedralize(self, input_file='heightmap.stl', output_file='heightmap_tet.msh'):
        """Perform tetrahedralization on an STL file and save the result."""
        if not os.path.isfile(input_file):
            raise FileNotFoundError(f"File {input_file} not found.")

        imesh = meshio.read(input_file)
        V = imesh.points
        F = imesh.cells_dict.get("triangle", [])

        if len(F) == 0:
            raise ValueError(f"No triangular faces found in {input_file}.")

        # Create TetGen mesher object
        mesher = tg.TetGen(V, F)

        try:
            # Perform tetrahedralization
            Vtg, Ctg = mesher.tetrahedralize(
                order=1,
                mindihedral=5.0,
                minratio=1.0
            )
        except RuntimeError as e:
            raise RuntimeError("Failed to tetrahedralize. May need to repair surface by making it manifold.") from e

        # Create and save the tetrahedral mesh
        omesh = meshio.Mesh(Vtg, [("tetra", Ctg)])
        meshio.write(output_file, omesh)
        self.tetrahedral_mesh = omesh
        print(f"Tetrahedral mesh saved to {output_file}")

    # Visualization
    def update_plotly_figure(self):
        """Update the Plotly figure with the current heightmap."""
        if self.heightmap is None:
            return
        heightmap_safe = np.nan_to_num(self.heightmap, nan=0.0, posinf=1.0, neginf=0.0)
        size_x, size_y = heightmap_safe.shape
        x = np.linspace(-1, 1, size_x)
        y = np.linspace(-1, 1, size_y)
        x, y = np.meshgrid(x, y)

        # Clear the figure before updating to avoid duplicates
        self.fig_widget.data = []  # Clear previous data
        self.fig_widget.add_surface(z=heightmap_safe, x=x, y=y, colorscale='Viridis')

        self.fig_widget.update_layout(
            title='Heightmap',
            autosize=True,
            scene=dict(
                zaxis=dict(title='Height'),
                xaxis=dict(title='X Axis'),
                yaxis=dict(title='Y Axis')
            ),
            margin=dict(l=65, r=50, b=65, t=90)
        )

    def display_interactive_controls(self, stl_filename='heightmap.stl', mesh_filename='heightmap_tet.msh'):
        """Display interactive controls and update the heightmap accordingly."""
        # Create interactive sliders
        amplitude_slider = widgets.FloatSlider(
            value=self.amplitude,
            min=0.1,
            max=2.0,
            step=0.1,
            description='Amplitude:',
            continuous_update=False
        )
        size_x_slider = widgets.IntSlider(
            value=self.size_x,
            min=10,
            max=200,
            step=10,
            description='Size X:',
            continuous_update=False
        )
        size_y_slider = widgets.IntSlider(
            value=self.size_y,
            min=10,
            max=200,
            step=10,
            description='Size Y:',
            continuous_update=False
        )
        noise_function_dropdown = widgets.Dropdown(
            options=[
                'Fractal Brownian Motion', 
                'Perlin Noise', 
                'Beckmann', 
                'Blinn-Phong', 
                'GGX', 
                'Sine Wave', 
                'Square Wave',
                'Mandelbrot'
            ],
            value='Fractal Brownian Motion',
            description='Noise Function:'
        )

        # Sliders for Fractal Brownian Motion parameters
        octaves_slider = widgets.IntSlider(
            value=self.noise_params.get('octaves', 4),
            min=1,
            max=10,
            step=1,
            description='Octaves:',
            continuous_update=False
        )
        persistence_slider = widgets.FloatSlider(
            value=self.noise_params.get('persistence', 0.5),
            min=0.1,
            max=1.0,
            step=0.1,
            description='Persistence:',
            continuous_update=False
        )
        lacunarity_slider = widgets.FloatSlider(
            value=self.noise_params.get('lacunarity', 2.0),
            min=1.0,
            max=5.0,
            step=0.1,
            description='Lacunarity:',
            continuous_update=False
        )

        # Slider for alpha parameter in Beckmann and GGX distributions
        alpha_slider = widgets.FloatSlider(
            value=self.noise_params.get('alpha', 0.5),
            min=0.01,
            max=1.0,
            step=0.01,
            description='Alpha (Roughness):',
            continuous_update=False
        )

        # Slider for n parameter in Blinn-Phong distribution
        n_slider = widgets.IntSlider(
            value=self.noise_params.get('n', 20),
            min=1,
            max=100,
            step=1,
            description='Shininess n:',
            continuous_update=False
        )

        # Slider for exponent to adjust peak sharpness
        exponent_slider = widgets.FloatSlider(
            value=self.noise_params.get('exponent', 1.0),
            min=0.1,
            max=5.0,
            step=0.1,
            description='Exponent:',
            continuous_update=False
        )

        # Sliders for Sine Wave and Square Wave parameters
        frequency_slider = widgets.FloatSlider(
            value=self.noise_params.get('frequency', 10.0),
            min=1.0,
            max=20.0,
            step=1.0,
            description='Frequency:',
            continuous_update=False
        )
        amplitude_slider_wave = widgets.FloatSlider(
            value=self.noise_params.get('amplitude', 1.0),
            min=0.1,
            max=5.0,
            step=0.1,
            description='Wave Amplitude:',
            continuous_update=False
        )

        # Sliders for Mandelbrot parameters
        mandelbrot_iterations_slider = widgets.IntSlider(
            value=self.noise_params.get('max_iterations', 100),
            min=10,
            max=1000,
            step=10,
            description='Max Iterations:',
            continuous_update=False
        )
        mandelbrot_threshold_slider = widgets.FloatSlider(
            value=self.noise_params.get('threshold', 2.0),
            min=1.0,
            max=4.0,
            step=0.1,
            description='Threshold:',
            continuous_update=False
        )

        DF_slider = widgets.FloatSlider(
            value=self.noise_params.get('D_f', 1.0),
            min=0.1,
            max=2.0,
            step=0.1,
            description='D_f:',
            continuous_update=False
        )
        gamma_slider = widgets.FloatSlider(
            value=self.noise_params.get('gamma', 1.0),
            min=0.1,
            max=2.0,
            step=0.1,
            description='Gamma:',
            continuous_update=False
        )
        M_slider = widgets.IntSlider(
            value=self.noise_params.get('M', 2),
            min=1,
            max=10,
            step=1,
            description='M:',
            continuous_update=False
        )

        N_max_slider = widgets.IntSlider(
            value=self.noise_params.get('N_max', 100),
            min=10,
            max=1000,
            step=10,
            description='N_max:',
            continuous_update=False
        )

        # Container to hold parameter sliders
        params_container = widgets.VBox()

        # Function to update parameter sliders based on selected noise function
        def update_params_sliders(*args):
            if noise_function_dropdown.value == 'Fractal Brownian Motion':
                params_container.children = [octaves_slider, persistence_slider, lacunarity_slider]
            elif noise_function_dropdown.value == 'Perlin Noise':
                params_container.children = []  # No additional parameters for Perlin Noise
            elif noise_function_dropdown.value == 'Beckmann' or noise_function_dropdown.value == 'GGX':
                params_container.children = [alpha_slider, exponent_slider]
            elif noise_function_dropdown.value == 'Blinn-Phong':
                params_container.children = [n_slider, exponent_slider]
            elif noise_function_dropdown.value == 'Sine Wave' or noise_function_dropdown.value == 'Square Wave':
                params_container.children = [frequency_slider, amplitude_slider_wave]
            elif noise_function_dropdown.value == 'Mandelbrot':
                params_container.children = [mandelbrot_iterations_slider, mandelbrot_threshold_slider, DF_slider, gamma_slider, M_slider, N_max_slider]
            else:
                params_container.children = []

        # Observe changes in the noise function dropdown
        noise_function_dropdown.observe(update_params_sliders, 'value')

        # Initialize parameter sliders
        update_params_sliders()

        # Define the update function
        def update_params(change):
            self.amplitude = amplitude_slider.value
            self.size_x = size_x_slider.value
            self.size_y = size_y_slider.value

            # Set the noise function based on the dropdown selection
            if noise_function_dropdown.value == 'Fractal Brownian Motion':
                self.noise_function = self.fbm_noise
                self.noise_params = {
                    'octaves': octaves_slider.value,
                    'persistence': persistence_slider.value,
                    'lacunarity': lacunarity_slider.value
                }
            elif noise_function_dropdown.value == 'Perlin Noise':
                self.noise_function = self.perlin_noise
                self.noise_params = {}
            elif noise_function_dropdown.value == 'Beckmann':
                self.noise_function = self.beckmann_noise
                self.noise_params = {
                    'alpha': alpha_slider.value,
                    'exponent': exponent_slider.value
                }
            elif noise_function_dropdown.value == 'GGX':
                self.noise_function = self.ggx_noise
                self.noise_params = {
                    'alpha': alpha_slider.value,
                    'exponent': exponent_slider.value
                }
            elif noise_function_dropdown.value == 'Blinn-Phong':
                self.noise_function = self.blinn_noise
                self.noise_params = {
                    'n': n_slider.value,
                    'exponent': exponent_slider.value
                }
            elif noise_function_dropdown.value == 'Sine Wave':
                self.noise_function = self.sine_wave
                self.noise_params = {
                    'frequency': frequency_slider.value,
                    'amplitude': amplitude_slider_wave.value
                }
            elif noise_function_dropdown.value == 'Square Wave':
                self.noise_function = self.square_wave
                self.noise_params = {
                    'frequency': frequency_slider.value,
                    'amplitude': amplitude_slider_wave.value
                }
            elif noise_function_dropdown.value == 'Mandelbrot':
                self.noise_function = self.mandelbrot_noise
                self.noise_params = {
                    'max_iterations': mandelbrot_iterations_slider.value,
                    'threshold': mandelbrot_threshold_slider.value,
                    'D_f': DF_slider.value,
                    'gamma': gamma_slider.value,
                    'M': M_slider.value,
                    'N_max': N_max_slider.value
                }
            else:
                self.noise_function = self.fbm_noise
                self.noise_params = {}

            # Regenerate heightmap and update the figure
            self.generate_heightmap()
            self.update_plotly_figure()

        # Attach the update_params function to widget events
        for widget_control in [
            amplitude_slider, size_x_slider, size_y_slider,
            octaves_slider, persistence_slider, lacunarity_slider,
            alpha_slider, n_slider,
            frequency_slider, amplitude_slider_wave, exponent_slider,
            mandelbrot_iterations_slider, mandelbrot_threshold_slider,
            DF_slider, gamma_slider, M_slider, N_max_slider,
            noise_function_dropdown
        ]:
            widget_control.observe(update_params, 'value')

        # Create the "Save Mesh" button
        save_mesh_button = widgets.Button(
            description='Save Mesh',
            button_style='success'
        )
        save_output = widgets.Output()

        # Define the function to save the mesh when the button is clicked
        def on_save_mesh_button_clicked(b):
            with save_output:
                save_output.clear_output()
                print("Saving mesh...")
                self.heightmap_to_closed_mesh()
                self.repair_mesh()
                self.save_mesh_to_stl(stl_filename)
                self.tetrahedralize(stl_filename, mesh_filename)
                print(f"Mesh saved to {stl_filename}")
                print(f"Tetrahedral mesh saved to {mesh_filename}")

        # Attach the function to the button click event
        save_mesh_button.on_click(on_save_mesh_button_clicked)

        # Create the UI layout
        ui = widgets.VBox([
            amplitude_slider,
            size_x_slider,
            size_y_slider,
            noise_function_dropdown,
            params_container,
            save_mesh_button,
            save_output
        ])

        # Initial update
        self.generate_heightmap()
        self.update_plotly_figure()

        # Display the interactive widgets and Plotly figure
        display(ui, self.fig_widget)

In [ ]:
# Create an instance of the generator with default parameters
generator = HeightmapMeshGenerator()

# Display interactive controls
generator.display_interactive_controls()